In [1]:
# Import all libraries
import os
import argparse
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms


In [2]:
# Data loader
train_loader = torch.utils.data.DataLoader(
	datasets.CIFAR10('../../data/lab1/imgFolders', train=True, download=True,
		transform=transforms.ToTensor()),
	batch_size=4, shuffle=True)

num_epoch = 5

Files already downloaded and verified


In [3]:
class BasicBlock(nn.Module):
    #expansion = 1
    def __init__(self, inplanes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        
        out += residual
        out = self.relu(out)

        return out


In [4]:
class ResNet(nn.Module):
    
    def __init__(self, block, layers, num_classes=10):
        self.input_channels = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,bias=False) # input ch = 3, output ch = 64 
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        #self.avgpool = nn.AvgPool2d(2)
        self.fc = nn.Linear(4096, num_classes)
        
    def _make_layer(self, block, target_output_channels, blocks, stride=1):
        layers = []
#         layers.append(block(self.input_channels, target_output_channels, stride))
        self.input_channels = target_output_channels
        for i in range(0, blocks):
            layers.append(block(self.input_channels, target_output_channels))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        #x = self.avgpool(x)
#         pdb.set_trace()
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x



In [5]:
model = ResNet(BasicBlock, [2]).cuda()
print(model)
bce_loss = nn.CrossEntropyLoss(size_average = False)
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

ResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d

In [6]:
def train(epoch):
    train_loss = 0
    model.train()
    
    for batch_idx, (data, true_labels) in enumerate(train_loader):
        data = Variable(data).cuda()
        true_labels = Variable(true_labels).cuda()
        optimizer.zero_grad()
        prediction = model(data)
        
#         print(prediction.size())
#         print(true_labels.size())
        loss = bce_loss(prediction,true_labels)
        
        loss.backward()
        
        train_loss += loss.data[0]
        
        optimizer.step()
        
    print("===> Epoch: {} Average loss: {:.4f}".format(
        epoch, train_loss / len(train_loader.dataset)))   

In [7]:
for epoch in range(1, num_epoch):
    train(epoch)

===> Epoch: 1 Average loss: 1.3260
===> Epoch: 2 Average loss: 0.9915
===> Epoch: 3 Average loss: 0.8535
===> Epoch: 4 Average loss: 0.7625
